## Calibration executer

Auteurs : Léa Drolet-Roy

Création : 2025-04-04
Dernière modification :

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import glob

import pyswarms as ps
from pyswarms.utils.plotters import plot_cost_history

from Calibration import Calibration
from Charucoboard import Charucoboard
from Checkerboard import Checkerboard

np.set_printoptions(precision=4, suppress=True)

In [ ]:
CHECKERBOARD = (9,4)
square_size = 112
marker_size = 86
image_size = (1080,1920)

aruco_name = cv2.aruco.DICT_4X4_100

cams = ['c1', 'c2', 'c3', 'c5', 'c6', 'c7', 'c8']
time_threshold = 15

path = r'C:\Users\LEA\Desktop\Poly\Trampo\video_test'
calib_dir = r'C:\Users\LEA\Desktop\Poly\Trampo\video_test\corners_found'

In [ ]:
# create Charurco board object
Board = Charucoboard(CHECKERBOARD, square_size, marker_size, aruco_name)

# validate board creation
Board.create_and_save_new_board()

# create Calibration object
Calib = Calibration(Board)

In [ ]:
Calib.saveImagesBoard(path, 'mp4')

In [ ]:
# Try setting an initial camera matrix
h, w = image_size
focal_length = 2000  # Approximate focal length
camera_matrix_init = np.array([
    [focal_length, 0, w / 2],
    [0, focal_length, h / 2],
    [0, 0, 1]], dtype=np.float32)

ret, C, S, D, K = Board.calibrate_intrinsics(calib_dir, cams, image_size, camera_matrix_init)

K = np.array(K)
np.savez('Intrinsics_K_trampo.npz', K)

D = np.array(D)
np.savez('Intrinsics_D_trampo.npz', D)

In [ ]:
K = np.load('Intrinsics_K_trampo.npz')['arr_0']
D = np.load('Intrinsics_D_trampo.npz')['arr_0']

print(f'Intrinsics: \n{K[0]} \n{K[1]} \n{K[2]} \n{K[3]} \n{K[4]} \n{K[5]}')

In [ ]:
Calib.saveStereoData(path, cams)

with open('stereo_data.pkl', 'rb') as f:
    stereo_images = pickle.load(f)

In [ ]:
path_stereo = os.path.join(path, 'stereo')

for camL in cams:
    
    Left_Params = {}
    Left_Params['Intrinsic'] = K[cams.index(camL)]
    Left_Params['Distortion'] = D[cams.index(camL)]

    for camR in cams[cams.index(camL)+1:]:
        print(camL, camR)

        Left_corners = [stereo_images['Corners'][i] for i in range(0, len(stereo_images['Corners']), 2) if stereo_images['Camera'][i] == camL[-1] and stereo_images['Camera'][i+1] == camR[-1]]
        Right_corners = [stereo_images['Corners'][i] for i in range(1, len(stereo_images['Corners']), 2) if stereo_images['Camera'][i-1] == camL[-1] and stereo_images['Camera'][i] == camR[-1]]
        
        Left_ids = [stereo_images['Ids'][i] for i in range(0, len(stereo_images['Corners']), 2) if stereo_images['Camera'][i] == camL[-1] and stereo_images['Camera'][i+1] == camR[-1]]
        Right_ids = [stereo_images['Ids'][i] for i in range(1, len(stereo_images['Corners']), 2) if stereo_images['Camera'][i-1] == camL[-1] and stereo_images['Camera'][i] == camR[-1]]
        
        Right_Params = {}
        
        if len(Left_corners) >= 1 and len(Right_corners) >= 1:

            Right_Params['Intrinsic'] = K[cams.index(camR)]
            Right_Params['Distortion'] = D[cams.index(camR)]
            
            ret, Stereo_Params = Calib.StereoCalibration(Left_Params, Right_Params, Left_corners, Left_ids, Right_corners, Right_ids)
            if ret:
                print('Transformation Matrix:')
                print(Stereo_Params['Transformation'])

                Calib.SaveParameters(camL, camR, Stereo_Params, Left_Params, Right_Params)
            else:
                    print('Not enough corners', '\n')
        else:
            print('Not enough images', '\n')

In [ ]:
c1_c2 = np.load('c1_c2_parameters.npz')['Transformation']
c1_c5 = np.load('c1_c5_parameters.npz')['Transformation']
c1_c6 = np.load('c1_c6_parameters.npz')['Transformation']
c1_c8 = np.load('c1_c8_parameters.npz')['Transformation']
c2_c3 = np.load('c2_c3_parameters.npz')['Transformation']
c2_c6 = np.load('c2_c6_parameters.npz')['Transformation']
# c3_c5 = np.load('c3_c5_parameters.npz')['Transformation']
# c5_c8 = np.load('c5_c8_parameters.npz')['Transformation']
c6_c7 = np.load('c6_c7_parameters.npz')['Transformation']
c7_c8 = np.load('c7_c8_parameters.npz')['Transformation']

Tcam1 = np.eye((4))
Tcam6 = c1_c6
Tcam5 = c1_c5
Tcam2 = c1_c2
Tcam3 = Tcam2 @ c2_c3
Tcam7 = Tcam6 @ c6_c7
Tcam8 = c1_c8

projMat = [Tcam1, Tcam2, Tcam3, Tcam5, Tcam6, Tcam7, Tcam8]

for mat in projMat:
    print(mat)

In [ ]:
## PSO for extrinsic parameters (with Bundle adjustment)
# Constants
num_cameras = 7
num_params_per_cam = 6
total_params = num_cameras * num_params_per_cam
n_particles = 60  # Number of particles
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7} #{'c1': 2.05, 'c2': 2.05, 'w': 0.729} # PSO Hyperparameters

init_params = []
for mat in projMat:
    rvec, _ = cv2.Rodrigues(mat[0:3,0:3])
    rvec = rvec.squeeze()
    params = [rvec[0], rvec[1], rvec[2], mat[0,3], mat[1,3], mat[2,3]]
    init_params.extend(params)

init_params = np.array(init_params, dtype=np.float64)
init_pos = np.tile(init_params, (n_particles,1))
print(init_params.shape)
print(init_pos.shape)

# Define parameter bounds
lower_bounds = [-2*np.pi, -2*np.pi, -2*np.pi, -5000, -5000, -5000]
upper_bounds = [2*np.pi, 2*np.pi, 2*np.pi, 5000, 5000, 6000]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))
print(param_bounds[0].shape)

print(np.all(param_bounds[0] <= init_pos[0]))
print(np.all(init_pos[0] <= param_bounds[1]))

In [ ]:
board = Board.type

def fun(params):
    errors = np.empty((n_particles,))
    params = np.array(params)

    for n in range(n_particles):
        projMat = np.empty((num_cameras+1, 3, 4))
        projMat[0] = np.hstack((np.eye((3)), np.zeros((3,1))))

        for cam_idx in range(num_cameras):
            cam_params = params[n][cam_idx * num_params_per_cam : (cam_idx + 1) * num_params_per_cam]
            r1, r2, r3, t1, t2, t3 = cam_params
            rvec = np.array([r1, r2, r3])
            R, _ = cv2.Rodrigues(rvec)
            t = np.array([t1, t2, t3]).reshape((3,1))
            projMat[cam_idx+1] = np.hstack((R, t))  #[cam_idx+1]

        RMSE = {c:[] for c in range(6)}
        # Loop on stereo images checkerboard points
        for i in range(0, len(stereo_images['Camera']) - 1, 2):
            j = i+1 # stereo image is the next one

            pts1_im = stereo_images['Charuco_Corners'][i].squeeze()
            pts2_im = stereo_images['Charuco_Corners'][j].squeeze()

            c1 = cams.index(f'c{stereo_images['Camera'][i]}')
            c2 = cams.index(f'c{stereo_images['Camera'][j]}')

            undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2)  # Shape (2, N)
            undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2)  # Shape (2, N)

            if board == 'charuco':
                Lids = stereo_images['Ids'][i].squeeze()
                Rids = stereo_images['Ids'][j].squeeze()

                obj_pts, img_pts_l, img_pts_r, common_ids = Board.getObjectImagePoints(undist_pts1, Lids, undist_pts2, Rids)
                img_pts_l, img_pts_r = img_pts_l.squeeze(), img_pts_r.squeeze()

                ids_to_keepL = [list(Lids).index(t) for t in common_ids]
                ids_to_keepR = [list(Rids).index(t) for t in common_ids]
                pts1_im, pts2_im = pts1_im[ids_to_keepL], pts2_im[ids_to_keepR]

            elif board == 'checker':
                img_pts_l, img_pts_r = undist_pts1, undist_pts2

            # Perform triangulation
            pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], img_pts_l.T, img_pts_r.T)
            points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
            points_3d = points_3d.T  # Shape (N, 3)

            # Compute RMSE for both cameras
            rmse1 = Calib.compute_rmse(pts1_im, points_3d, projMat[c1], K[c1], D[c1])
            rmse2 = Calib.compute_rmse(pts2_im, points_3d, projMat[c2], K[c2], D[c2])

            RMSE[c1].append(rmse1)
            RMSE[c2].append(rmse2)

        # Per-camera mean RMSE
        rmse_mean = []
        for _, rmse in RMSE.items():
            rmse_mean.append(np.mean(rmse))

        errors[n] = np.mean(rmse_mean) #+ np.max(RMSE)   # OPTIONAL: add max error to cost function
    
    return errors